In [2]:
from unsloth import FastModel
import torch
import torch.nn as nn
from datasets import load_dataset
import re
from trl import GRPOConfig, GRPOTrainer
from transformers import (
    GPT2Model,
    GPT2Tokenizer,
    GPT2PreTrainedModel,
    GPT2Config,
    Trainer,
    TrainingArguments,
    AutoModelForCausalLM,
    TextStreamer,
 AutoTokenizer
)
import random
from typing import Dict, List
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
from tqdm import tqdm
from datasets import Dataset as HFDataset
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import datetime
import time
from sklearn.preprocessing import StandardScaler
import pickle
from transformers import GPT2LMHeadModel, GPT2Tokenizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 07-03 18:51:01 [__init__.py:256] Automatically detected platform cuda.


In [4]:
df = pd.read_csv('summaries.csv')
random.seed(40)
labels_p = np.random.binomial(size=len(df), n=1, p= 0.5)
labels_b = np.random.binomial(size=len(df), n=1, p= 0.5)

In [5]:
def create_txt(prompt, model, human, label):
    if label == 1:
        return '''Which of the following summaries does a better job of summarizing the most important points in the given forum post, without including unimportant or irrelevant details? Judge based on accuracy, coverage, and coherence.
    Post: {''' + prompt +  '''} 
    Summary A: {''' + model + '''} 
    
    Summary B: {''' + human + '''}

    
    Which summary is better? Respond with only <A> or <B>.
    -------------------------------------------------------
    \n\n
    '''
    else:
        return '''Which of the following summaries does a better job of summarizing the most important points in the given forum post, without including unimportant or irrelevant details? Judge based on accuracy, coverage, and coherence.
    Post: {''' + prompt +  '''} 
    Summary A: {''' + human + '''} 
    
    Summary B: {''' + model + '''}

    
    Which summary is better? Respond with only <A> or <B>.
    -------------------------------------------------------
    \n\n
    '''

In [7]:
# Format Data for GPT2
text_p = []
text_b = []
ml = 0 
for i, row in df.iterrows():
    # Process the prompt
    prompt = row['Prompt']
    index = prompt.find('Post')
    if index != -1: #check if substring exists
        prompt = prompt[index+5:]
        prompt = prompt[: len(prompt)-8]
    text_p.append(create_txt(prompt, row['Pessimism'], row['Human'], labels_p[i]))
    text_b.append(create_txt(prompt, row['Base'], row['Human'], labels_b[i]))
    ml = max(ml, len(text_p[-1].split(' ')))

In [8]:
print(len(text_p), len(text_b))

300 300


In [9]:
with open('../whome/Downloads/summ_pess.txt', 'w') as f:
    for line in text_p:
        f.write("%s\n" % line)

In [10]:
with open('../whome/Downloads/summ_base.txt', 'w') as f:
    for line in text_b:
        f.write("%s\n" % line)

In [16]:
res_p = []
i = 0
with open('../whome/Downloads/output_p.txt', 'r') as file:
    for line in file:
        # Process each line here
        if 'A' in line:
            if labels_p[i] == 1:
                res_p.append(1)
            else:
                res_p.append(0)
            i+=1
        if 'B' in line:
            if labels_p[i] == 0:
                res_p.append(1)
            else:
                res_p.append(0)
            i += 1
i = 0 
res_b = []
with open('../whome/Downloads/output_b.txt', 'r') as file:
    for line in file:
        if 'A' in line:
            if labels_b[i] == 1:
                res_b.append(1)
            else:
                res_b.append(0)

            i+=1
        if 'B' in line:
            if labels_b[i] == 0:
                res_b.append(1)
            else:
                res_b.append(0)
            i+=1

In [17]:
print(len(res_p), len(res_b))

300 300


In [18]:
print(np.sum(res_p)/len(res_p), np.sum(res_b)/len(res_b))

0.53 0.49
